Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 4

## Catch up, if needed
- [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2/portfolio-project/ds6), then choose your dataset, and [submit this form](https://forms.gle/nyWURUg65x1UTRNV9), due yesterday at 3:59pm Pacific.
- Submit predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file.) The competition closes today at 3:59pm. Every student should make at least one submission that scores at least 60% accuracy (above the majority class baseline).

## Assignment
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your final predictions to our Kaggle competition. Optionally, go to **My Submissions**, and _"you may select up to 1 submission to be used to count towards your final leaderboard score."_ The competition closes today at 3:59pm.
- [ ] Add comments and Markdown to your notebook. Clean up your code.
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Try combining xgboost early stopping, cross-validation, & hyperparameter optimization, with [the "original" (non scikit-learn) xgboost API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.cv).
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

#### Try stacking multiple submissions!

Here's some code you can use:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [2]:
import pandas as pd
import numpy as np

In [4]:
# Import train features and labels as train data
train = pd.merge(pd.read_csv('C://Users/ajaco/Desktop/train_features.csv'),
                 pd.read_csv('C://Users/ajaco/Desktop/train_labels.csv'))

# Import test features
test = pd.read_csv('C:/Users/ajaco/Desktop/test_features.csv')

# Import sample_submission file for creating submission later
sample_submission = pd.read_csv('C://Users/ajaco/Desktop/sample_submission.csv')


train.shape, test.shape

((59400, 41), (14358, 40))

In [5]:
def wrangle(X):

    # Prevent SettingWithCopyWarning
    X = X.copy()

    # Treat zeros as nulls, convert near zeros to zero
    X['latitude'] = X['latitude'].replace(-2e-08, 0)

    # Replace suspicious zeroes with nans, then create new
    # column with binary variable for whether it was missing
    # will impute values later
    cols_with_zeros = ['longitude', 'latitude', 'construction_year',
                       'gps_height', 'population']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()

    # Convert date_recorded to datetime object
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True) 

    # Extract year, month, and day from date_recorded and drop date_recorded
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded') 

    # New column time_from_construction_to_inspection
    X['years_til_inspection'] = X['year_recorded'] - X['construction_year'] 
    X['years_missing'] = X['years_til_inspection'].isnull()
  

    # quantity group is a duplicate of quantity so drop
    X = X.drop(columns=['quantity_group', 'scheme_management', 'num_private', 
                        'recorded_by', 'payment', 'waterpoint_type'])

    return X

In [6]:
# Wrangle train and test data
train = wrangle(train)
test = wrangle(test)

In [37]:
import category_encoders as ce
from sklearn.feature_selection import SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline


In [43]:
# Select features and create X matrix and y vector
target = 'status_group'
features = train.columns.drop(['id', target]).tolist()
X_train = train[features]
y_train = train[target]

# Create pipeline; ordinal encoding,
# simple imputing w median,
# RFClass model n_estimators=500
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=500, max_depth=9, random_state=42, n_jobs=-1)
)

# Get scores and print
scores = cross_val_score(pipeline, X_train, y_train, cv=5)
print(scores)

[0.76188873 0.74396095 0.74924242 0.74654882 0.75500926]


In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import validation_curve

In [44]:
depth = range(1, 40, 2)
train_scores, val_scores = validation_curve(
    pipeline, X_train, y_train,
    param_name='randomforestclassifier__max_depth',
    param_range=depth, scoring='accuracy',
    cv=3,
    n_jobs=-1
)

plt.plot(depth, np.mean(train_scores, axis=1), color='b', label='training error')
plt.plot(depth, np.mean(val_scores, axis=1), color='r', label='validation error')
plt.xlabel('Depth of Trees')
plt.ylabel('Accuracy Score')
plt.legend();

KeyboardInterrupt: 

In [18]:
from sklearn.model_selection import RandomizedSearchCV

In [24]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    #SelectKBest(f_regression),
    RandomForestClassifier(n_estimators=100, n_jobs=-1)
)

pipeline.fit(X_train, y_train)

pipeline.score(X_train, y_train)

0.9999494949494949

In [39]:
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    SelectKBest(),
    RandomForestClassifier(n_jobs=-1)
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'selectkbest__k': range(1, len(X_train.columns)+1),
    'randomforestclassifier__n_estimators': range(50, 500),
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=5,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   51.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed:  6.5min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  7.4min finished


In [40]:
search.best_params_

{'simpleimputer__strategy': 'mean',
 'selectkbest__k': 32,
 'randomforestclassifier__n_estimators': 427}

In [41]:
search.best_score_

0.7957239057239057

In [36]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

cross_val_score(pipeline, X_train, y_train, cv=2, scoring='accuracy')

array([0.79714488, 0.79628944])

In [57]:
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    SelectKBest(),
    RandomForestClassifier(n_jobs=-1)
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'selectkbest__k': range(1, len(X_train.columns)+1),
    'randomforestclassifier__n_estimators': range(50, 500),
    'randomforestclassifier__max_depth': [5, 10, 20, 50, None],
    'randomforestclassifier__min_samples_leaf': range(1, 50, 5)
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 13.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 16.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 19.7min finished


In [49]:
search.best_params_

{'simpleimputer__strategy': 'median',
 'selectkbest__k': 42,
 'randomforestclassifier__n_estimators': 400}

In [50]:
search.best_score_

0.8093434343434344

In [54]:
import xgboost as xgb

In [55]:
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    SelectKBest(),
    xgb.XGBClassifier(n_jobs=-1)
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'selectkbest__k': range(1, len(X_train.columns)+1),
    'xgbclassifier__n_estimators': range(100, 500),
    'xgbclassifier__max_depth': range(1,10),
    'xgbclassifier__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3]
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(X_train, y_train);

print('Best Score: ', search.best_score_)
print('Best Parameters: ', search.best_params_)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 22.2min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed: 28.3min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed: 32.0min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed: 37.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 42.0min finished


Best Score:  0.8017003367003367
Best Parameters:  {'xgbclassifier__n_estimators': 228, 'xgbclassifier__max_depth': 9, 'xgbclassifier__learning_rate': 0.1, 'simpleimputer__strategy': 'mean', 'selectkbest__k': 23}
